In [1]:
import pandas as pd

In [2]:
# 메뉴정보 20251017_TOTAL_MENU.csv

menu_df = pd.read_csv('./20251017_TOTAL_MENU.csv')

In [3]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54625 entries, 0 to 54624
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   식당ID     54625 non-null  int64 
 1   식당명      54625 non-null  object
 2   메뉴       54606 non-null  object
 3   가격원문     54599 non-null  object
 4   대표여부     54625 non-null  object
 5   가격범위     54480 non-null  object
 6   테마       54140 non-null  object
 7   온도       54072 non-null  object
 8   카테고리     54068 non-null  object
 9   주재료      54062 non-null  object
 10  맵기(O/X)  54054 non-null  object
dtypes: int64(1), object(10)
memory usage: 4.6+ MB


In [4]:
# 음식점 목록 20251016_서울시_음식점_목록_GPS.csv

store_df = pd.read_csv('./20251016_서울시_음식점_목록_GPS.csv')

In [5]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7713 entries, 0 to 7712
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          7713 non-null   int64  
 1   상세URL       7713 non-null   object 
 2   가게          7713 non-null   object 
 3   주소          7713 non-null   object 
 4   카테고리        7702 non-null   object 
 5   리본개수        7713 non-null   int64  
 6   레드리본 선정     7713 non-null   object 
 7   서울 2025 선정  7713 non-null   object 
 8   라벨          1483 non-null   object 
 9   이미지URL      7713 non-null   object 
 10  이미지파일       7713 non-null   object 
 11  소개          7712 non-null   object 
 12  X좌표         7713 non-null   float64
 13  Y좌표         7713 non-null   float64
dtypes: float64(2), int64(2), object(10)
memory usage: 843.7+ KB


In [8]:
%pip install -q pandas openai requests beautifulsoup4


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.13 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import mykeys


In [2]:
mykeys.setOsEnv()

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.
mykey => set HUGGINGFACEHUB_API_TOKEN is done
mykey => set OPENAI_API_KEY is done
mykey => set LANGCHAIN_API_KEY is done
mykey => set LANGCHAIN_HUB_API_KEY is done
mykey => set GOOGLE_API_KEY is done
mykey => set GOOGLE_CSE_ID is done
mykey => set UPSTAGE_API_KEY is done
mykey => set COHERE_API_KEY is done
mykey => set JINA_API_KEY is done
mykey => set ANTHROPIC_API_KEY is done
mykey => set DEEPL_API_KEY is done
mykey => set TAVILY_API_KEY is done
mykey => set TOGETHER_API_KEY is done


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openai
import json
import os
import time
import concurrent.futures

# --- 1. 설정: API 키 및 모델 설정 ---

# (중요!)
# 스크립트 실행 전, 터미널에서 API 키를 설정해야 합니다.
# export OPENAI_API_KEY="sk-..."
try:
  client = openai.OpenAI()
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  exit()

GPT_API_NAME = "gpt-4.1"
# (중요!) 동시 작업 스레드 수. 
# 너무 높으면 API 속도 제한이나 IP 차단 위험이 있습니다. 5~10이 적절합니다.
MAX_WORKERS = 5

# (중요!) 스크래핑 차단을 피하기 위한 필수 User-Agent
REQUEST_HEADERS = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

# --- 2. 헬퍼 함수 정의 ---

def scrape_url(url):
  """
  상세 URL을 스크래핑하고 정제된 텍스트를 반환합니다.
  (참고: 이 함수는 매우 일반적이며, 실제 '태깅' 정보는 
   CSS 선택자(selector)를 지정해야 정확히 추출할 수 있습니다.)
  """
  try:
    response = requests.get(url, headers=REQUEST_HEADERS, timeout=10)
    response.raise_for_status() # 오류가 있으면 예외 발생

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # script, style, nav, footer 등 불필요한 태그 제거
    for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
      tag.decompose()
      
    # 모든 텍스트를 공백으로 연결하여 추출
    text = soup.get_text(separator=' ', strip=True)
    
    # LLM의 토큰 제한을 피하기 위해 텍스트 길이 제한 (약 4000자)
    max_chars = 4000
    if len(text) > max_chars:
      text = text[:max_chars] + "... (내용이 너무 길어 생략됨)"
      
    return text
  
  except requests.RequestException as e:
    print(f"  > [오류] URL 스크래핑 실패 (URL: {url}): {e}")
    return "스크래핑 실패: URL에 접근할 수 없습니다."

def format_prompt(store_row, menu_group, scraped_text):
  """
  LLM에 전달할 프롬프트를 생성합니다.
  """
  
  # 1. store_df 정보 (Pandas Series -> JSON)
  store_json = store_row.to_json(force_ascii=False, indent=2)
  
  # 2. menu_df 정보 (DataFrame -> JSON)
  # (요청하신 필수 컬럼만 선택)
  menu_columns = ['메뉴', '가격원문', '테마', '온도', '카테고리', '주재료', '맵기(O/X)']
  # menu_group에 해당 컬럼이 있는지 확인 후 추출
  valid_menu_cols = [col for col in menu_columns if col in menu_group.columns]
  menu_json = menu_group[valid_menu_cols].to_json(orient='records', force_ascii=False, indent=2)

  # 3. 시스템 프롬프트 (AI의 역할 정의)
  system_prompt = """
  당신은 레스토랑 정보를 분석하고 요약하는 최고의 '데이터 분석가'입니다.
  당신의 임무는 3가지 다른 소스(가게 정보, 메뉴 정보, 웹사이트 텍스트)를
  종합적으로 분석하여, 한국어로 유용하고 상세한 요약문을 생성하는 것입니다.
  """
  
  # 4. 유저 프롬프트 (실제 데이터와 작업 지시)
  # (마크다운을 제거하고 f-string만 사용)
  user_prompt = f"""
  다음은 하나의 레스토랑에 대한 3가지 데이터 소스입니다.
  
  [소스 1: 가게 기본 정보 (store_df)]
  {store_json}
  
  [소스 2: 가게 메뉴 정보 (menu_df)]
  {menu_json}
  
  [소스 3: 가게 상세 URL 스크랩 텍스트]
  (여기에는 사용자들이 남긴 '태깅'이나 '기타 유용한정보'가 포함되어 있습니다.)
  {scraped_text}
  
  ---
  [작업 요청]
  위 3가지 소스를 면밀히 분석하여, 이 가게의 특징을 상세히 요약해주세요.
  
  [요약문에 반드시 포함될 내용]
  1.  [소스 1]의 모든 정보 (가게명, 주소, 카테고리, 리본/라벨 정보)
  2.  [소스 2]의 핵심 메뉴, 가격대, 메뉴의 테마, 온도, 주재료, 맵기 특징
  3.  [소스 3]에서 발견되는 사용자 태그, 분위기, 특별한 서비스 등 유용한 정보
  
  [요약문 (한국어)]
  """
  
  return system_prompt, user_prompt


def get_llm_summary(system_prompt, user_prompt):
  """
  GPT-4.1 API를 호출하여 요약문을 받아옵니다.
  """
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
      ],
      temperature=0.5 # 요약 작업이므로 창의성(T)을 낮춤
    )
    summary = response.choices[0].message.content
    return summary
  
  except Exception as e:
    print(f"  > [오류] LLM API 호출 실패: {e}")
    return f"요약 생성 실패: {e}"

def process_restaurant(store_row, menu_group):
  """
  레스토랑 1개에 대한 전체 파이프라인 (스크래핑 + LLM)
  (이 함수가 병렬로 실행됩니다.)
  """
  
  store_id = store_row['id']
  store_name = store_row['가게']
  url = store_row['상세URL']
  
  print(f"--- 작업 시작 (ID: {store_id}, 가게: {store_name}) ---")
  
  # 1. 스크래핑
  print(f"  > [1/3] 스크래핑 시작: {url}")
  scraped_text = scrape_url(url)
  
  # 2. 프롬프트 생성
  print(f"  > [2/3] 프롬프트 생성 중...")
  system_prompt, user_prompt = format_prompt(store_row, menu_group, scraped_text)
  
  # 3. LLM 요약
  print(f"  > [3/3] LLM ({GPT_API_NAME}) 요약 요청 중...")
  summary_text = get_llm_summary(system_prompt, user_prompt)
  
  print(f"--- 작업 완료 (ID: {store_id}) ---")
  
  return (store_id, store_name, summary_text)

In [7]:
# --- 3. 메인 실행 함수 ---
def main():
  # (★★★ 1. count 변수 설정 ★★★)
  # (경고: 7713개 전체를 실행하면 17시간+ 및 막대한 API 비용이 발생합니다)
  COUNT_TO_PROCESS = 1 # (테스트할 개수. 0 또는 7713으로 설정 시 전체 실행)

  # (★★★ 2. 실제 데이터 로드 ★★★)
  # (Charlie님이 이 파일 경로를 실제 경로로 수정해야 합니다)
  try:
    print("데이터 로드 중: store_df, menu_df ...")
    # (파일 경로는 실제 환경에 맞게 수정하세요)
    store_df = pd.read_csv('./20251016_서울시_음식점_목록_GPS.csv')
    menu_df = pd.read_csv('./20251017_TOTAL_MENU.csv')
    print(f"로드 완료: store_df {store_df.shape}, menu_df {menu_df.shape}")
  except FileNotFoundError as e:
    print(f"[오류] 파일 로드 실패: {e}")
    print("스크립트와 동일한 폴더에 'store.csv'와 'menu.csv' 파일이 있는지 확인하세요.")
    return # 파일 없으면 중단

  # (★★★ 3. count 변수값에 맞게 선택 ★★★)
  if COUNT_TO_PROCESS > 0 and COUNT_TO_PROCESS < len(store_df):
    print(f"총 {len(store_df)}개 중 {COUNT_TO_PROCESS}개만 샘플로 선택합니다.")
    store_df_sample = store_df.head(COUNT_TO_PROCESS).copy()
  else:
    print(f"총 {len(store_df)}개 전체를 처리합니다.")
    store_df_sample = store_df.copy()
    
  # (이하 로직은 이전과 동일)

  # menu_df를 식당ID별로 미리 그룹화 (효율성)
  menu_groups = menu_df.groupby('식당ID') 
  
  # 결과를 저장할 리스트
  summaries_results = []
  
  print(f"총 {len(store_df_sample)}개 레스토랑 요약 작업을 시작합니다. (최대 작업자 수: {MAX_WORKERS})")
  
  # ThreadPoolExecutor를 사용한 병렬 처리
  with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # (id, row)로 반복하며 작업(future)을 제출
    future_to_id = {}
    for _, store_row in store_df_sample.iterrows():
      store_id = store_row['id']
      
      try:
        # 해당 식당의 메뉴 그룹 가져오기
        menu_group = menu_groups.get_group(store_id)
      except KeyError:
        # 메뉴 정보가 없는 식당 (빈 DataFrame 생성)
        print(f"  > [경고] ID {store_id}는 메뉴 정보(menu_df)가 없습니다.")
        menu_group = pd.DataFrame(columns=menu_df.columns)
        
      # 병렬 작업 큐에 (작업 함수, 인자1, 인자2)를 넣음
      future = executor.submit(process_restaurant, store_row, menu_group)
      future_to_id[future] = store_id

    # 작업이 완료되는 순서대로 결과 처리
    for future in concurrent.futures.as_completed(future_to_id):
      store_id = future_to_id[future]
      try:
        # result = (store_id, store_name, summary_text) 튜플 반환
        result = future.result()
        
        # (★★★ 수정된 부분 ★★★)
        # 튜플(result) 전체가 아닌, 요약문(result[2])을 직접 출력해야
        # \n (줄바꿈)이 예쁘게 적용됩니다.
        
        store_id_val = result[0] # (ID)
        store_name_val = result[1] # (가게명)
        summary_text = result[2] # (요약문)

        print("\n" + "="*60)
        print(f"✅ [로그] 요약 완료 (ID: {store_id_val}, 가게: {store_name_val})")
        print("="*60)
        # 요약문 텍스트(string)를 직접 출력
        print(summary_text) 
        print("="*60 + "\n")
        
        # (원래 코드)
        summaries_results.append(result)
      except Exception as e:
        print(f"!!! 치명적 오류 (ID: {store_id}): {e}")
        summaries_results.append((store_id, "N/A", f"작업 실패: {e}"))

  print(f"\n--- 모든 작업 완료 ---")
  
  # 결과(리스트)를 DataFrame으로 변환
  summary_df = pd.DataFrame(summaries_results, columns=['id', '가게', 'LLM요약'])
  
  # 원본 store_df와 병합
  final_df = pd.merge(store_df, summary_df[['id', 'LLM요약']], on='id', how='left')

  #print(final_df)
  
  # 파일로 저장
  final_df.to_csv("restaurant_summaries.csv", index=False, encoding='utf-8-sig')
  print("결과가 'restaurant_summaries.csv' 파일로 저장되었습니다.")


if __name__ == "__main__":
  main()

데이터 로드 중: store_df, menu_df ...
로드 완료: store_df (7713, 14), menu_df (54625, 11)
총 7713개 중 1개만 샘플로 선택합니다.
총 1개 레스토랑 요약 작업을 시작합니다. (최대 작업자 수: 5)
--- 작업 시작 (ID: 30639, 가게: 다이닝마) ---
  > [1/3] 스크래핑 시작: https://www.bluer.co.kr/restaurants/30639
  > [2/3] 프롬프트 생성 중...
  > [3/3] LLM (gpt-4.1) 요약 요청 중...
--- 작업 완료 (ID: 30639) ---

✅ [로그] 요약 완료 (ID: 30639, 가게: 다이닝마)
[다이닝마] 상세 요약

1. **기본 정보**
- **가게명:** 다이닝마
- **주소:** 서울특별시 강남구 언주로152길 8 (신사동) 유일빌딩 2층
- **카테고리:** 일반중식
- **리본 개수:** 3개
- **레드리본 선정:** N
- **서울 2025 선정:** Y (라벨: 서울 2025 선정)
- **상세 URL:** [https://www.bluer.co.kr/restaurants/30639](https://www.bluer.co.kr/restaurants/30639)
- **위치 안내:** 도산공원 사거리에서 BMW 강남전시장 골목으로 진입 후 우측
- **전화번호:** 02-518-7400
- **영업시간:** 월~토 11:30~21:30 (브레이크타임 14:30~17:30, 마지막 주문 14:00/21:00), 일요일 및 명절 당일 휴무
- **개업일:** 2019년 11월 11일
- **주차/서비스:** 발레파킹 가능
- **이미지:** ![다이닝마 이미지](https://www.bluer.co.kr/images/es_f1fa131ef8324c8ba133315f9acc71ec.png)

2. **메뉴 및 가격대**
- **주요 메뉴 및 가격**
  - 다이닝마코스 (A): 290,000원
  - 다이닝마